In [1]:
## Standard libraries
import os
from copy import deepcopy

## Imports for plotting
import matplotlib.pyplot as plt
plt.set_cmap('cividis')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.set()

## tqdm for loading bars
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

## Torchvision
import torchvision
from torchvision.datasets import STL10
from torchvision import transforms

# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Import tensorboard
%load_ext tensorboard

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../saved_models/tutorial17"
# In this notebook, we use data loaders with heavier computational processing. It is recommended to use as many
# workers as possible in a data loader, which corresponds to the number of CPU cores
NUM_WORKERS = os.cpu_count()

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)
print("Number of workers:", NUM_WORKERS)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  # Remove the CWD from sys.path while we load stuff.


Device: cuda:0
Number of workers: 2


In [2]:
import urllib.request
from urllib.error import HTTPError
from torchvision import transforms
import os
# Github URL where saved models are stored for this tutorial
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial17/"
# Files to download
pretrained_files = ["SimCLR.ckpt", "ResNet.ckpt",
                    "tensorboards/SimCLR/events.out.tfevents.SimCLR",
                    "tensorboards/classification/ResNet/events.out.tfevents.ResNet"]
pretrained_files += [f"LogisticRegression_{size}.ckpt" for size in [10, 20, 50, 100, 200, 500]]

In [3]:
class ContrastiveTransformations(object):
    
    def __init__(self, base_transforms, n_views=2):
        self.base_transforms = base_transforms
        self.n_views = n_views
        
    def __call__(self, x):
        return [self.base_transforms(x) for i in range(self.n_views)]

In [4]:
contrast_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
                                          transforms.RandomResizedCrop(size=96),
                                          transforms.RandomApply([
                                              transforms.ColorJitter(brightness=0.5, 
                                                                     contrast=0.5, 
                                                                     saturation=0.5, 
                                                                     hue=0.1)
                                          ], p=0.8),
                                          transforms.RandomGrayscale(p=0.2),
                                          transforms.GaussianBlur(kernel_size=9),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.5,), (0.5,))
                                         ])

In [5]:
import pandas as pd
df = pd.read_csv("../input/sdss-images/data.csv")
df.iloc[0]["image"]

'image_0000.jpg'

In [6]:
df['class'].value_counts()

GALAXY    4998
STAR      4152
QSO        850
Name: class, dtype: int64

In [7]:
import numpy as np
import torch.nn.functional as nnf
from torch.utils.data.dataset import Dataset
from skimage import io as skio
from skimage import transform
from torchvision import transforms as T

class SDSSDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_file)
        label_dict = {'GALAXY':0,'STAR':1,'QSO':2}
        self.df['class'] = self.df['class'].apply(lambda x : label_dict[x])
        print(self.df.head(2))
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.df.iloc[idx]["image"])
        image = skio.imread(img_name).reshape(3,128,128).astype(np.float32)
        torch_tensor = torch.from_numpy(image).unsqueeze(0)
        
        out = nnf.interpolate(torch_tensor, size=(96, 96), mode='nearest').squeeze(0)
        
        image = out.cpu().detach().numpy().reshape(3,96,96)
#         print(image.shape)
        label = self.df.iloc[idx]["class"]
        return image,label

In [8]:
image = skio.imread("../input/sdss-images/images (1)/images/image_0000.jpg")
image.reshape(3,128,128).shape

(3, 128, 128)

In [9]:
import shutil
import pandas as pd
balanced_set = pd.read_csv("../input/sdss-images/equal_splits.csv")
print(balanced_set.columns)
balanced_images = list(balanced_set[balanced_set.columns[1]])
balanced_images.extend(list(balanced_set[balanced_set.columns[2]]))
balanced_images.extend(list(balanced_set[balanced_set.columns[3]]))

Index(['Unnamed: 0', 'trimmed traininng  qalaxy', 'trimmed traininng  qsd',
       'trimmed traininng  star'],
      dtype='object')


In [10]:
!rm -rf balanced_images/*
!mkdir balanced_images

In [11]:
for img in balanced_images:
    shutil.copy("../input/sdss-images/images (1)/images/"+img,"./balanced_images/"+img)

In [12]:
df = pd.read_csv("../input/sdss-images/data.csv")
balanced_df = df.loc[df['image'].isin(balanced_images)].reset_index().drop(['index'],axis=1)

In [13]:
from sklearn.model_selection import train_test_split
balanced_train,balanced_test = train_test_split(balanced_df,test_size=0.2,stratify=balanced_df['class'])
balanced_train.to_csv("balanced_train.csv",index=False),balanced_test.to_csv("balanced_test.csv",index=False)

(None, None)

In [15]:
class SimCLR(pl.LightningModule):
    
    def __init__(self, hidden_dim, lr, temperature, weight_decay, max_epochs=500):
        super().__init__()
        self.save_hyperparameters()
        assert self.hparams.temperature > 0.0, 'The temperature must be a positive float!'
        # Base model f(.)
        self.convnet = torchvision.models.resnet18(num_classes=4*hidden_dim)  # Output of last linear layer
        # The MLP for g(.) consists of Linear->ReLU->Linear 
        self.convnet.fc = nn.Sequential(
            self.convnet.fc,  # Linear(ResNet output, 4*hidden_dim)
            nn.ReLU(inplace=True),
            nn.Linear(4*hidden_dim, hidden_dim)
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), 
                                lr=self.hparams.lr, 
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                            T_max=self.hparams.max_epochs,
                                                            eta_min=self.hparams.lr/50)
        return [optimizer], [lr_scheduler]
        
    def info_nce_loss(self, batch, mode='train'):
        imgs, _ = batch
        imgs = list([i.cpu().detach().numpy() for i in imgs])
#         print(len(imgs),imgs[0]

#         imgs = torch.cat(imgs, dim=0)
        
        imgs = torch.tensor(imgs).to(device)
        # Encode all images
#         print(imgs.shape)
        feats = self.convnet(imgs.float())
        # Calculate cosine similarity
        cos_sim = F.cosine_similarity(feats[:,None,:], feats[None,:,:], dim=-1)
        # Mask out cosine similarity to itself
        self_mask = torch.eye(cos_sim.shape[0], dtype=torch.bool, device=cos_sim.device)
        cos_sim.masked_fill_(self_mask, -9e15)
        # Find positive example -> batch_size//2 away from the original example
        pos_mask = self_mask.roll(shifts=cos_sim.shape[0]//2, dims=0)
        # InfoNCE loss
        cos_sim = cos_sim / self.hparams.temperature
        nll = -cos_sim[pos_mask] + torch.logsumexp(cos_sim, dim=-1)
        nll = nll.mean()
        
        # Logging loss
        self.log(mode+'_loss', nll)
        # Get ranking position of positive example
        comb_sim = torch.cat([cos_sim[pos_mask][:,None],  # First position positive example
                              cos_sim.masked_fill(pos_mask, -9e15)], 
                             dim=-1)
        sim_argsort = comb_sim.argsort(dim=-1, descending=True).argmin(dim=-1)
        # Logging ranking metrics
        self.log(mode+'_acc_top1', (sim_argsort == 0).float().mean())
        self.log(mode+'_acc_top5', (sim_argsort < 5).float().mean())
        self.log(mode+'_acc_mean_pos', 1+sim_argsort.float().mean())
        
        return nll
        
    def training_step(self, batch, batch_idx):
        
        return self.info_nce_loss(batch, mode='train')
        
    def validation_step(self, batch, batch_idx):
        self.info_nce_loss(batch, mode='val')
        

In [16]:
model = SimCLR(hidden_dim=128,lr=5e-4,temperature=0.07,weight_decay=1e-4,max_epochs=50)
model.load_state_dict(torch.load("../input/simclr-ml701/simclr_resnet.pth"))

<All keys matched successfully>

In [17]:
def train_simclr(batch_size, max_epochs=500, **kwargs):
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, 'SimCLR'),
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=max_epochs,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode='max', monitor='val_acc_top5'),
                                    LearningRateMonitor('epoch')])
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, 'SimCLR.ckpt')
    if os.path.isfile(pretrained_filename):
        print(f'Found pretrained model at {pretrained_filename}, loading...')
        model = SimCLR.load_from_checkpoint(pretrained_filename) # Automatically loads the model with the saved hyperparameters
    else:
        train_loader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                                       drop_last=True, pin_memory=True, num_workers=NUM_WORKERS)
        val_loader = data.DataLoader(dataset, batch_size=batch_size, shuffle=False, 
                                     drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
        pl.seed_everything(42) # To be reproducable
        model = SimCLR(max_epochs=max_epochs, **kwargs)
        trainer.fit(model, train_loader, val_loader)
        model = SimCLR.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    return model

In [18]:
class LogisticRegression(pl.LightningModule):
    
    def __init__(self, feature_dim, num_classes, lr, weight_decay, max_epochs=100):
        super().__init__()
        self.save_hyperparameters()
        # Mapping from representation h to classes
        self.model = nn.Linear(feature_dim, num_classes)

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), 
                                lr=self.hparams.lr, 
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, 
                                                      milestones=[int(self.hparams.max_epochs*0.6), 
                                                                  int(self.hparams.max_epochs*0.8)], 
                                                      gamma=0.1)
        return [optimizer], [lr_scheduler]
        
    def _calculate_loss(self, batch, mode='train'):
        feats, labels = batch
        preds = self.model(feats)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log(mode + '_loss', loss)
        self.log(mode + '_acc', acc)
        return loss        
        
    def training_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode='train')
        
    def validation_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode='val')
        
    def test_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode='test')
    
    def forward(self,batch):
        return self.model(batch)

In [19]:
img_transforms = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5,), (0.5,))])

train_img_data = SDSSDataset("./balanced_train.csv","./balanced_images")
test_img_data = SDSSDataset("./balanced_test.csv","./balanced_images")
print("Number of training examples:", len(train_img_data))
print("Number of test examples:", len(test_img_data))

   class           image
0      2  image_7444.jpg
1      0  image_1163.jpg
   class           image
0      1  image_0846.jpg
1      0  image_9084.jpg
Number of training examples: 1560
Number of test examples: 390


<!-- Next, we implement a small function to encode all images in our datasets. The output representations are then used as inputs to the Logistic Regression model. -->

In [20]:
@torch.no_grad()
def prepare_data_features(model, dataset):
    # Prepare model
    network = deepcopy(model.convnet)
    network.fc = nn.Identity()  # Removing projection head g(.)
    network.eval()
    network.to(device)
    
    # Encode all images
    data_loader = data.DataLoader(dataset, batch_size=64, num_workers=NUM_WORKERS, shuffle=False, drop_last=False)
    feats, labels = [], []
    for batch_imgs, batch_labels in tqdm(data_loader):
        batch_imgs = batch_imgs.to(device)
        batch_feats = network(batch_imgs)
        feats.append(batch_feats.detach().cpu())
        labels.append(batch_labels)
    
    feats = torch.cat(feats, dim=0)
    labels = torch.cat(labels, dim=0)
    
    # Sort images by labels
    labels, idxs = labels.sort()
    feats = feats[idxs]
    
    return data.TensorDataset(feats, labels)

In [21]:
train_feats_simclr = prepare_data_features(model, train_img_data)
test_feats_simclr = prepare_data_features(model, test_img_data)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [22]:
def train_logreg(batch_size, train_feats_data, test_feats_data, model_suffix, max_epochs=100, **kwargs):
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, "LogisticRegression"),
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=max_epochs,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode='max', monitor='val_acc'),
                                    LearningRateMonitor("epoch")],
                         enable_progress_bar=False,
                         check_val_every_n_epoch=10)
    trainer.logger._default_hp_metric = None
    
    # Data loaders
    train_loader = data.DataLoader(train_feats_data, batch_size=batch_size, shuffle=True, 
                                   drop_last=False, pin_memory=True, num_workers=0)
    test_loader = data.DataLoader(test_feats_data, batch_size=batch_size, shuffle=False, 
                                  drop_last=False, pin_memory=True, num_workers=0)

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"LogisticRegression_{model_suffix}.ckpt")
    if os.path.isfile(pretrained_filename):
        print(f"Found pretrained model at {pretrained_filename}, loading...")
        model = LogisticRegression.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything(42)  # To be reproducable
        model = LogisticRegression(**kwargs)
        trainer.fit(model, train_loader, test_loader)
        model = LogisticRegression.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)

    # Test best model on train and validation set
#     train_result = trainer.test(model, train_loader, verbose=False)
#     test_result = trainer.test(model, test_loader, verbose=False)
#     result = {"train": train_result[0]["test_acc"], "test": test_result[0]["test_acc"]}
        
    return trainer,model#, result

In [23]:
results = {}
trainer,model = train_logreg(batch_size=64,
                                    train_feats_data=train_feats_simclr,
                                    test_feats_data=test_feats_simclr,
                                    model_suffix="any",
                                    feature_dim=train_feats_simclr.tensors[0].shape[1],
                                    num_classes=3,
                                    lr=1e-3,
                                    weight_decay=1e-3)
    

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1896: PossibleUserWarning: The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


In [24]:
test_loader = data.DataLoader(test_feats_simclr, batch_size=1, shuffle=False, 
                                  drop_last=False, pin_memory=True, num_workers=0)

In [25]:
preds = list()
labels = list()
for i in test_loader:
    preds.append(model.forward(i[0]))
    labels.append(i[1])

In [26]:
import numpy as np
pred_argmax = [np.argmax(list(p.detach().cpu().numpy()[0])) for p in preds]
labels_argmax = [l.detach().cpu().numpy()[0] for l in labels]

In [27]:
from sklearn.metrics import classification_report
classification_report(pred_argmax,labels_argmax).split("\n")

['              precision    recall  f1-score   support',
 '',
 '           0       0.91      0.89      0.90       133',
 '           1       0.80      0.80      0.80       130',
 '           2       0.76      0.78      0.77       127',
 '',
 '    accuracy                           0.82       390',
 '   macro avg       0.82      0.82      0.82       390',
 'weighted avg       0.82      0.82      0.82       390',
 '']

## Baseline

In [28]:
class ResNet(pl.LightningModule):

    def __init__(self, num_classes, lr, weight_decay, max_epochs=100):
        super().__init__()
        self.save_hyperparameters()
        self.model = torchvision.models.resnet18(num_classes=num_classes)

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), 
                                lr=self.hparams.lr, 
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, 
                                                      milestones=[int(self.hparams.max_epochs*0.7), 
                                                                  int(self.hparams.max_epochs*0.9)], 
                                                      gamma=0.1)
        return [optimizer], [lr_scheduler]

    def _calculate_loss(self, batch, mode='train'):
        imgs, labels = batch
        preds = self.model(imgs)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log(mode + '_loss', loss)
        self.log(mode + '_acc', acc)
        return loss

    def training_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode='train')

    def validation_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode='val')

    def test_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode='test')
    def forward(self,batch):
        return self.model(batch)

In [29]:
train_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
                                       transforms.RandomResizedCrop(size=96, scale=(0.8, 1.0)),
                                       transforms.RandomGrayscale(p=0.2),
                                       transforms.GaussianBlur(kernel_size=9, sigma=(0.1, 0.5)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5,), (0.5,))
                                       ])



train_img_aug_data = SDSSDataset("./balanced_train.csv","./balanced_images")
test_img_aug_data = SDSSDataset("./balanced_test.csv","./balanced_images")

   class           image
0      2  image_7444.jpg
1      0  image_1163.jpg
   class           image
0      1  image_0846.jpg
1      0  image_9084.jpg


In [30]:
def train_resnet(batch_size, max_epochs=100, **kwargs):
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, "ResNet"),
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=max_epochs,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                                    LearningRateMonitor("epoch")],
                         check_val_every_n_epoch=2)
    trainer.logger._default_hp_metric = None
    
    # Data loaders
    
    train_loader = data.DataLoader(train_img_aug_data, batch_size=batch_size, shuffle=True, 
                                   drop_last=True, pin_memory=True, num_workers=NUM_WORKERS)
    test_loader = data.DataLoader(test_img_data, batch_size=batch_size, shuffle=False, 
                                  drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, "ResNet.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model at %s, loading..." % pretrained_filename)
        model = ResNet.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything(42) # To be reproducable
        model = ResNet(**kwargs)
        trainer.fit(model, train_loader, test_loader)
        print(trainer.checkpoint_callback.best_model_path)
        model = ResNet.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)

    # Test best model on validation set
#     train_result = trainer.test(model, train_loader, verbose=False)
#     val_result = trainer.test(model, test_loader, verbose=False)
#     result = {"train": train_result[0]["test_acc"], "test": val_result[0]["test_acc"]}
        
    return trainer,model

# Resnet fine tune

In [35]:
def get_net():
    resnet = torchvision.models.resnet34(pretrained=True)
    
    # Substitute the FC output layer
    resnet.fc = torch.nn.Linear(resnet.fc.in_features, 10)
    torch.nn.init.xavier_uniform_(resnet.fc.weight)
    return resnet


import time

def train(net, train_dataloader, valid_dataloader, criterion, optimizer, scheduler=None, epochs=10, device='cpu', checkpoint_epochs=10):
    start = time.time()
    print(f'Training for {epochs} epochs on {device}')
    
    for epoch in range(1,epochs+1):
        print(f"Epoch {epoch}/{epochs}")
        
        net.train()  # put network in train mode for Dropout and Batch Normalization
        train_loss = torch.tensor(0., device=device)  # loss and accuracy tensors are on the GPU to avoid data transfers
        train_accuracy = torch.tensor(0., device=device)
        for X, y in train_dataloader:
            X = X.to(device)
            y = y.to(device)
            preds = net(X)
            loss = criterion(preds, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                train_loss += loss * train_dataloader.batch_size
                train_accuracy += (torch.argmax(preds, dim=1) == y).sum()
        
        if valid_dataloader is not None:
            net.eval()  # put network in train mode for Dropout and Batch Normalization
            valid_loss = torch.tensor(0., device=device)
            valid_accuracy = torch.tensor(0., device=device)
            with torch.no_grad():
                for X, y in valid_dataloader:
                    X = X.to(device)
                    y = y.to(device)
                    preds = net(X)
                    loss = criterion(preds, y)

                    valid_loss += loss * valid_dataloader.batch_size
                    valid_accuracy += (torch.argmax(preds, dim=1) == y).sum()
        
        if scheduler is not None: 
            scheduler.step()
            
        print(f'Training loss: {train_loss/len(train_dataloader.dataset):.2f}')
        print(f'Training accuracy: {100*train_accuracy/len(train_dataloader.dataset):.2f}')
        
        if valid_dataloader is not None:
            print(f'Valid loss: {valid_loss/len(valid_dataloader.dataset):.2f}')
            print(f'Valid accuracy: {100*valid_accuracy/len(valid_dataloader.dataset):.2f}')
        
        if epoch%checkpoint_epochs==0:
            torch.save({
                'epoch': epoch,
                'state_dict': net.state_dict(),
                'optimizer': optimizer.state_dict(),
            }, './checkpoint.pth.tar')
        
        print()
        
        if (100*train_accuracy/len(train_dataloader.dataset))>97:
            break
    end = time.time()
    print(f'Total training time: {end-start:.1f} seconds')
    return net

train_img_aug_data = SDSSDataset("./balanced_train.csv","./balanced_images")
test_img_aug_data = SDSSDataset("./balanced_test.csv","./balanced_images")
train_loader = data.DataLoader(train_img_aug_data, batch_size=64, shuffle=True, 
                               drop_last=True, pin_memory=True, num_workers=NUM_WORKERS)
test_loader = data.DataLoader(test_img_data, batch_size=64, shuffle=False, 
                              drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr, weight_decay, epochs = 1e-5, 5e-4, 20

net = get_net().to(device)

# Standard CrossEntropy Loss for multi-class classification problems
criterion = torch.nn.CrossEntropyLoss()

# params_1x are the parameters of the network body, i.e., of all layers except the FC layers
params_1x = [param for name, param in net.named_parameters() if 'fc' not in str(name)]
optimizer = torch.optim.Adam([{'params':params_1x}, {'params': net.fc.parameters(), 'lr': lr*10}], lr=lr, weight_decay=weight_decay)

net = train(net, train_loader, test_loader, criterion, optimizer, None, epochs, device)

   class           image
0      2  image_7444.jpg
1      0  image_1163.jpg
   class           image
0      1  image_0846.jpg
1      0  image_9084.jpg


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

Training for 20 epochs on cuda
Epoch 1/20
Training loss: 2.29
Training accuracy: 26.92
Valid loss: 1.62
Valid accuracy: 49.23

Epoch 2/20
Training loss: 0.71
Training accuracy: 76.60
Valid loss: 0.98
Valid accuracy: 72.56

Epoch 3/20
Training loss: 0.37
Training accuracy: 87.18
Valid loss: 0.87
Valid accuracy: 74.10

Epoch 4/20
Training loss: 0.25
Training accuracy: 90.83
Valid loss: 0.85
Valid accuracy: 75.38

Epoch 5/20
Training loss: 0.19
Training accuracy: 93.46
Valid loss: 0.82
Valid accuracy: 76.67

Epoch 6/20
Training loss: 0.12
Training accuracy: 95.51
Valid loss: 0.84
Valid accuracy: 77.18

Epoch 7/20
Training loss: 0.10
Training accuracy: 96.54
Valid loss: 0.84
Valid accuracy: 77.69

Epoch 8/20
Training loss: 0.08
Training accuracy: 97.31
Valid loss: 0.83
Valid accuracy: 77.44

Total training time: 27.8 seconds


In [36]:
net.eval()
labels = []
testlabels = []
test_loader = data.DataLoader(test_img_aug_data, batch_size=1, shuffle=False, 
                                  drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
for inputs,labels1 in tqdm(test_loader):
    inputs = inputs.to(device)
    testlabels.append(labels1.detach().cpu().numpy()[0])
    labels1 = torch.tensor(labels1).to(device)
    outputs = net(inputs)
    _, predictions = torch.max(outputs, 1)
    predictions = predictions.to("cpu")
    labels.extend(predictions.numpy())
    

  0%|          | 0/390 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == "__main__":


In [37]:
import numpy as np

from sklearn.metrics import classification_report
classification_report(testlabels,labels).split("\n")

['              precision    recall  f1-score   support',
 '',
 '           0       0.88      0.89      0.89       130',
 '           1       0.74      0.67      0.70       130',
 '           2       0.71      0.76      0.73       130',
 '',
 '    accuracy                           0.77       390',
 '   macro avg       0.77      0.77      0.77       390',
 'weighted avg       0.77      0.77      0.77       390',
 '']

# Resnet scratch

In [38]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,models
from tqdm import tqdm_notebook as tqdm

class ResidualBlock(nn.Module):
    def __init__(self,in_channels,out_channels,stride=1,kernel_size=3,padding=1,bias=False):
        super(ResidualBlock,self).__init__()
        self.cnn1 =nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(True)
        )
        self.cnn2 = nn.Sequential(
            nn.Conv2d(out_channels,out_channels,kernel_size,1,padding,bias=False),
            nn.BatchNorm2d(out_channels)
        )
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Sequential()
            
    def forward(self,x):
        residual = x
        x = self.cnn1(x)
        x = self.cnn2(x)
        x += self.shortcut(residual)
        x = nn.ReLU(True)(x)
        return x
    

class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18,self).__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=2,stride=2,padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.block2 = nn.Sequential(
            nn.MaxPool2d(1,1),
            ResidualBlock(64,64),
            ResidualBlock(64,64,2)
        )
        
        self.block3 = nn.Sequential(
            ResidualBlock(64,128),
            ResidualBlock(128,128,2)
        )
        
        self.block4 = nn.Sequential(
            ResidualBlock(128,256),
            ResidualBlock(256,256,2)
        )
        self.block5 = nn.Sequential(
            ResidualBlock(256,512),
            ResidualBlock(512,512,2)
        )
        
        self.avgpool = nn.AvgPool2d(2)
        # vowel_diacritic
        self.fc1 = nn.Linear(512,3)
        
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x1 = self.fc1(x)
        return x1
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [39]:
import numpy as np
import torch.nn.functional as nnf
from torch.utils.data.dataset import Dataset
from skimage import io as skio
from skimage import transform
from torchvision import transforms as T

class SDSSDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_file)
        label_dict = {'GALAXY':0,'STAR':1,'QSO':2}
        self.df['class'] = self.df['class'].apply(lambda x : label_dict[x])
        print(self.df.head(2))
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.df.iloc[idx]["image"])
        image = skio.imread(img_name).reshape(3,128,128).astype(np.float32)
        torch_tensor = torch.from_numpy(image).unsqueeze(0)
        
        out = nnf.interpolate(torch_tensor, size=(64, 64), mode='nearest').squeeze(0)
        
        image = out.cpu().detach().numpy().reshape(3,64,64)
#         print(image.shape)
        label = self.df.iloc[idx]["class"]
        return image,label
train_img_aug_data = SDSSDataset("./balanced_train.csv","./balanced_images")
test_img_aug_data = SDSSDataset("./balanced_test.csv","./balanced_images")

   class           image
0      2  image_7444.jpg
1      0  image_1163.jpg
   class           image
0      1  image_0846.jpg
1      0  image_9084.jpg


In [40]:
model = ResNet18().to(device)
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=4e-4)
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=0.05)
criterion = nn.CrossEntropyLoss()
batch_size=32
epochs = 12 # original 50
model.train()
losses = []
accs = []
for epoch in range(epochs):
    train_loader = data.DataLoader(train_img_aug_data, batch_size=batch_size, shuffle=True, 
                                   drop_last=True, pin_memory=True, num_workers=NUM_WORKERS)
    test_loader = data.DataLoader(test_img_data, batch_size=batch_size, shuffle=False, 
                                  drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
    
    print('epochs {}/{} '.format(epoch+1,epochs))
    running_loss = 0.0
    running_acc = 0.0
    for idx, (inputs,labels1) in tqdm(enumerate(train_loader),total=len(train_loader)):
        inputs = inputs.to(device)
        labels = torch.tensor(labels1).to(device)
        
        optimizer.zero_grad()
        outputs1 = model(inputs.float())
        loss1 = criterion(outputs1.to(device),labels1.to(device))
        
        running_loss = loss1
        running_acc = (outputs1.argmax(1).to(device)==labels1.to(device)).float().mean()
        
        loss1.backward()
        optimizer.step()
    #scheduler.step()
    losses.append(running_loss/len(train_loader))
    accs.append(running_acc/(len(train_loader)))
    print('acc : {:.2f}%'.format(running_acc))
    
    print('loss : {:.4f}'.format(running_loss))
    if running_acc>0.98:
        break
torch.save(model.state_dict(), 'resnet34_50epochs_saved_weights.pth')

epochs 1/12 


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/48 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


acc : 0.91%
loss : 0.3475
epochs 2/12 


...

acc : 0.97%
loss : 0.0425
epochs 9/12 


  0%|          | 0/48 [00:00<?, ?it/s]

acc : 0.91%
loss : 0.1662
epochs 10/12 


  0%|          | 0/48 [00:00<?, ?it/s]

acc : 1.00%
loss : 0.0193


In [41]:

model.load_state_dict(torch.load('resnet34_50epochs_saved_weights.pth'))

<All keys matched successfully>

In [42]:
model.eval()
labels = []
testlabels = []
test_loader = data.DataLoader(test_img_aug_data, batch_size=1, shuffle=False, 
                                  drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
for inputs,labels1 in tqdm(test_loader):
    inputs = inputs.to(device)
    testlabels.append(labels1.detach().cpu().numpy()[0])
    labels1 = torch.tensor(labels1).to(device)
    outputs = model(inputs)
    _, predictions = torch.max(outputs, 1)
    predictions = predictions.to("cpu")
    labels.extend(predictions.numpy())
    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/390 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == "__main__":


In [43]:
import numpy as np

from sklearn.metrics import classification_report
classification_report(testlabels,labels).split("\n")

['              precision    recall  f1-score   support',
 '',
 '           0       0.97      0.92      0.94       130',
 '           1       0.97      0.74      0.84       130',
 '           2       0.75      0.97      0.85       130',
 '',
 '    accuracy                           0.87       390',
 '   macro avg       0.90      0.87      0.87       390',
 'weighted avg       0.90      0.87      0.87       390',
 '']

---

[![Star our repository](https://img.shields.io/static/v1.svg?logo=star&label=⭐&message=Star%20Our%20Repository&color=yellow)](https://github.com/phlippe/uvadlc_notebooks/)  If you found this tutorial helpful, consider ⭐-ing our repository.    
[![Ask questions](https://img.shields.io/static/v1.svg?logo=star&label=❔&message=Ask%20Questions&color=9cf)](https://github.com/phlippe/uvadlc_notebooks/issues)  For any questions, typos, or bugs that you found, please raise an issue on GitHub. 

---